In [1]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [2]:
from typing import TypedDict
from langgraph.graph import StateGraph


class InputState(TypedDict):
    question: str


class OutputState(TypedDict):
    answer: str


class OverallState(InputState, OutputState):
    pass

In [3]:
builder = StateGraph(OverallState, input_schema=InputState, output_schema=OutputState)

In [4]:
def agent_node(state: InputState):
    print("我是一个 AI Agent")
    return


def action_node(state: InputState):
    print("我是一个 Action")
    return {"answer": "我现在执行成功了"}

In [5]:
from langgraph.graph import START, END

builder.add_node(agent_node)
builder.add_node(action_node)

builder.add_edge(START, "agent_node")
builder.add_edge("agent_node", "action_node")
builder.add_edge("action_node", END)

graph = builder.compile()

In [6]:
graph.invoke({"question": "你好"})

我是一个 AI Agent
我是一个 Action


{'answer': '我现在执行成功了'}

In [8]:
def agent_node(state: InputState):
    print("我是一个 AI Agent")
    return {"question": state["question"]}


def action_node(state: InputState):
    print("我是一个 Action")
    return {"answer": f"我接收到的问题是：{state['question']}, 读取成功了！"}


builder = StateGraph(OverallState, input_schema=InputState, output_schema=OutputState)

builder.add_node(agent_node)
builder.add_node(action_node)

builder.add_edge(START, "agent_node")
builder.add_edge("agent_node", "action_node")
builder.add_edge("action_node", END)

graph = builder.compile()

In [10]:
graph.invoke({"question": "你好"})

我是一个 AI Agent
我是一个 Action


{'answer': '我接收到的问题是：你好, 读取成功了！'}

In [11]:
from langchain_openai.chat_models import ChatOpenAI


def llm_node(state: InputState):
    messages = [
        ("system", "你是一位乐于助人的智能小助理"),
        ("human", state["question"]),
    ]
    llm = ChatOpenAI(model="qwen-plus", temperature=0)

    response = llm.invoke(messages)

    return {"answer": response.content}

In [12]:
builder = StateGraph(OverallState, input_schema=InputState, output_schema=OutputState)

builder.add_node(llm_node)

builder.add_edge(START, "llm_node")
builder.add_edge("llm_node", END)

graph = builder.compile()

In [13]:
graph.invoke({"question": "你好，我用来测试"})

{'answer': '你好呀！测试什么内容呢？😊'}

In [14]:
graph.invoke({"question": "你好，请详细的介绍你自己"})

{'answer': '你好呀！我是Qwen3，是通义千问系列中效果最好的大语言模型 😊 作为你的智能小助理，我可以帮助你做很多事情呢！\n\n我可以帮你写故事、写公文、写邮件、写剧本等等，需要创作的时候随时找我哦。如果你需要分析数据或者处理一些信息，我也可以帮你搞定。编程方面我也不赖，写代码、调试、解释代码都能行。\n\n我还能表达观点，玩游戏，或者就某个话题展开讨论。如果你需要一个耐心的聊天伙伴，我随时都在。不过要记得，虽然我喜欢聊天，但也要注意合理使用，不要影响到学习和工作哦！\n\n说到学习，我可以帮助不同年龄段的学生，从作业辅导到学术写作都能提供帮助。对于职场人士，我也可以提供专业的建议和方案。\n\n最重要的是，我会始终遵循安全和伦理准则，确保我们的交流既愉快又有意义。虽然我很强，但我也在不断学习和进步中，有时候可能需要你的理解和耐心。\n\n希望我们能成为好朋友！有什么需要帮忙的，尽管告诉我哦！ 🌟'}

In [15]:
from typing import TypedDict
from langgraph.graph import StateGraph


class InputState(TypedDict):
    question: str
    llm_answer: str | None


class OutputState(TypedDict):
    answer: str


class OverallState(InputState, OutputState):
    pass


def llm_node(state: InputState):
    messages = [
        ("system", "你是一位乐于助人的智能小助理"),
        ("human", state["question"]),
    ]
    llm = ChatOpenAI(model="qwen-plus", temperature=0)

    response = llm.invoke(messages)

    return {"llm_answer": response.content}


def action_node(state: InputState):
    messages = [
        ("system", "无论你接收到什么语言的文本，请翻译成法语"),
        ("human", state["llm_answer"]),
    ]
    llm = ChatOpenAI(model="qwen-plus", temperature=0)

    response = llm.invoke(messages)

    return {"answer": response.content}


builder = StateGraph(OverallState, input_schema=InputState, output_schema=OutputState)

builder.add_node(llm_node)
builder.add_node(action_node)

builder.add_edge(START, "llm_node")
builder.add_edge("llm_node", "action_node")
builder.add_edge("action_node", END)

graph = builder.compile()

In [16]:
final_answer = graph.invoke({"question": "你好，请你详细的介绍一下你自己"})
print(final_answer["answer"])

Bonjour ! Je suis Qwen3, le modèle phare de la série Tongyi Qianwen, doté de fonctionnalités particulièrement puissantes. En tant que votre petit assistant intelligent, je peux mener avec vous des conversations en plusieurs tours, répondre à vos questions, et vous aider à créer des textes, comme rédiger des histoires, des documents officiels, des courriels, des scénarios, faire des raisonnements logiques ou programmer. Je peux également exprimer des opinions, jouer à des jeux, et même vous aider à traduire du contenu entre plusieurs langues.

Ma formation reposant sur des données extrêmement vastes, j'ai acquis des connaissances transversales dans divers domaines ainsi qu'une solide capacité de compréhension linguistique. Que ce soit des questions académiques, des connaissances générales liées à la vie quotidienne ou encore des potins sur le divertissement, je peux discuter avec vous ! De plus, je maîtrise particulièrement bien le chinois, ainsi que plusieurs autres langues.

Si vous a